In [6]:
#Combo_Top_10_total_price

import os
import pandas as pd
import re
import numpy as np

# ================= USER INPUT =================
COMBO_FILE   = input("Enter Combo Excel file path: ").strip()
PRICE_FILE   = input("Enter Price Excel file path: ").strip()
OUTPUT_FILE  = input("Enter Output Excel file path: ").strip()
INVESTMENT   = float(input("Enter investment per company (e.g. 50000): "))

#  Ensure old file is removed
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

# ================= LOAD PRICE DATA =================
price_df = pd.read_excel(PRICE_FILE, sheet_name=0)
price_df["DATE"] = pd.to_datetime(price_df["DATE"], format="%d-%b-%y")
price_df = price_df.set_index("DATE")

# ================= HELPER FUNCTIONS =================
def extract_date(col_name):
    match = re.search(r"\d{4}-\d{2}-\d{2}", col_name)
    if match:
        return pd.to_datetime(match.group())
    else:
        raise ValueError(f"Date not found in column name: {col_name}")

def nearest_date(index, target_date):
    return index[(abs(index - target_date)).argmin()]

# ================= PROCESS COMBO FILE =================
combo_xls = pd.ExcelFile(COMBO_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:
    for i, sheet in enumerate(combo_xls.sheet_names, start=1):

        combo_df = pd.read_excel(combo_xls, sheet_name=sheet)

        start_col = combo_df.columns[1]
        end_col   = combo_df.columns[2]

        start_date = extract_date(start_col)
        end_date   = extract_date(end_col)

        if start_date not in price_df.index:
            start_date = nearest_date(price_df.index, start_date)
        if end_date not in price_df.index:
            end_date = nearest_date(price_df.index, end_date)

        start_prices = price_df.loc[start_date]
        end_prices   = price_df.loc[end_date]

        output_rows = []

        for combo in combo_df["Combo"]:
            companies = combo.split(",")

            quantities = [
                INVESTMENT / float(start_prices[c])
                for c in companies
            ]

            start_total = int(round(
                sum(q * float(start_prices[c]) for q, c in zip(quantities, companies))
            ))

            end_total = int(round(
                sum(q * float(end_prices[c]) for q, c in zip(quantities, companies))
            ))

            output_rows.append({
                "Combo": combo,
                "start_price": start_total,
                "final_price": end_total
            })

        out_df = pd.DataFrame(output_rows)
        out_df.to_excel(writer, sheet_name=f"Week{i}", index=False)

print("\n Process completed successfully")
print(f" Output file created at: {OUTPUT_FILE}")


Enter Combo Excel file path: D:\week_qty\Combo_top_10_input_data.xlsx
Enter Price Excel file path: C:\swarupa\Trading Stratergies\50 comp data (input file).xlsx
Enter Output Excel file path: D:\week_qty\Combo_Top_10_total_price.xlsx
Enter investment per company (e.g. 50000): 90000

 Process completed successfully
 Output file created at: D:\week_qty\Combo_Top_10_total_price.xlsx


In [8]:
#Combo_Top_10_Gain_Loss

import pandas as pd
import numpy as np

# ===================== USER INPUT =====================
input_file = input("Enter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ===================== LOAD INPUT =====================
xls = pd.ExcelFile(input_file)

summary_rows = []
detailed_sheets = {}

# ===================== PROCESS EACH INPUT SHEET =====================
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)

    # ----- ADD CALCULATION COLUMNS -----
    df["Gain_Loss_Amt"] = df["final_price"] - df["start_price"]
    df["Gain_Amt"] = np.where(df["Gain_Loss_Amt"] > 0, df["Gain_Loss_Amt"], 0)
    df["Loss_Amt"] = np.where(df["Gain_Loss_Amt"] < 0, abs(df["Gain_Loss_Amt"]), 0)
    df["No_of_Gains"] = np.where(df["Gain_Loss_Amt"] > 0, 1, 0)
    df["No_of_Loss"] = np.where(df["Gain_Loss_Amt"] < 0, 1, 0)

    # ----- SELECT AND REORDER COLUMNS FOR DETAILED SHEET -----
    detailed_df = df[["Combo", "start_price", "final_price", "Gain_Loss_Amt", "Gain_Amt", "Loss_Amt"]]
    detailed_sheets[sheet_name] = detailed_df

    # ----- CREATE SHEET-WISE SUMMARY ROW -----
    summary_rows.append({
        "Sheet_Name": sheet_name,
        "No_of_Gains": int(df["No_of_Gains"].sum()),
        "No_of_Loss": int(df["No_of_Loss"].sum()),
        "Total_Gains_Amt": round(df["Gain_Amt"].sum(), 2),
        "Total_Loss_Amt": round(df["Loss_Amt"].sum(), 2),
        "Net_Gain_Loss_Amt": round(df["Gain_Amt"].sum() - df["Loss_Amt"].sum(), 2)
    })

# ---------- WRITE OUTPUT FILE WITH SUMMARY FIRST ----------
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:

    # Write summary sheet first
    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_excel(writer, sheet_name="Summary_All_Sheets", index=False)

    # Write all detailed sheets after
    for sheet_name, df in detailed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(" Output created ")


Enter input Excel file path:  D:\week_qty\Combo_Top_10_total_price.xlsx
Enter output Excel file path: D:\week_qty\Combo_Top_10_Gain_Loss.xlsx
 Output created: Summary sheet first, detailed sheets with selected columns


In [12]:
#Summary_With_Percentage_Columns

import pandas as pd

# ================= USER INPUT =================
input_file = input("Enter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ================= LOAD INPUT  =================
df = pd.read_excel(input_file)

# ================= CONSTANTS =================
CAPITAL = 360000 * 10
TOTAL_COMBOS = 10
TOTAL_WEEKS = len(df)   

# ================= ADD OUTPUT COLUMNS =================
df["%Returns"] = ((df["Net_Gain_Loss_Amt"] / CAPITAL) * 100).round(2)
df["%No_of_Gains"] = (df["No_of_Gains"] / TOTAL_COMBOS) * 100
df["%No_of_Loss"] = (df["No_of_Loss"] / TOTAL_COMBOS) * 100

# ================= OVERALL SUMMARY =================
overall_df = pd.DataFrame({
    "Metric": [
        "%Total_Returns",
        "%Total_No_of_Gains",
        "%Total_No_of_Loss"
    ],
    "Value": [
        round((df["Net_Gain_Loss_Amt"].sum() / (CAPITAL * TOTAL_WEEKS)) * 100,2),
        round((df["No_of_Gains"].sum() / (TOTAL_COMBOS * TOTAL_WEEKS)) * 100,2),
        round((df["No_of_Loss"].sum() / (TOTAL_COMBOS * TOTAL_WEEKS)) * 100,2)
    ]
})

# ================= WRITE OUTPUT =================
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="Weekly_Data", index=False)
    overall_df.to_excel(writer, sheet_name="Overall_Summary", index=False)

print("\n Calculation completed successfully.")
print(" Output file created at:", output_file)


Enter input Excel file path: D:\week_qty\Summary_All_Sheets_202.xlsx
Enter output Excel file path: D:\week_qty\Summary_With_Percentage_Columns.xlsx

 Calculation completed successfully.
 Output file created at: D:\week_qty\Summary_With_Percentage_Columns.xlsx
